# PDF text Extraction

ML6 use case. 20 pdf documents with product Safety hazard information.


Safety Data Sheet (SDS) information:

https://en.wikipedia.org/wiki/Safety_data_sheet

https://ekos.gda.pl/en/article/baza-wiedzy/safety-data-sheet-annex-ii.html

http://chemicalsafety.com/sds-search/

https://www.msdsonline.com/sds-search/

https://comply1.com/compliance/ghs/MSDS-SDS-Comparison

https://www.everysafeuse.com/chemparser/     - Commercial product that converts SDS into json - json output example


https://echa.europa.eu/-/guidance-on-the-compilation-of-safety-data-sheets


Assumptions:
      - all Safety Data Sheets are in English only.


### test file

In [1]:
InputFile = 'data/23114.pdf'

### Extract PDF file as one single text string

In [9]:
import PyPDF2 
  
import PyPDF2 
   
pdfFileObj = open(InputFile, 'rb') 
  
pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
  
print(f"Number of pages in file '{InputFile}' = {pdfReader.numPages}\n") 

pageObj = pdfReader.getPage(0) 
  
PdfText = pageObj.extractText() 

pdfFileObj.close()

print(PdfText)

Number of pages in file 'data/23114.pdf' = 10

   Material Safety Data Sheet 
  LUPEROX® P
      Product code: 051000
   Version 2.0
 Issued on: 07/11/2011
 Page: 1 / 10
     1. PRODUCT AND COMPANY IDENTIFICATION
  Company
  
Arkema Inc.
 900 First Avenue 
King of Prussia, Pennsylvania 19406 

 Functional Additives
  
Customer Service Telephone Number: 
(800) 331-7654 
(Monday through Friday, 8:30 AM to 5:30 PM EST) 
 Emergency Information
  
Transportation:
 CHEMTREC: (800) 424-9300 
(24 hrs., 7 days a week) 
Medical: 
Rocky Mountain Poison Center:
 (866) 767-5089 
(24 hrs., 7 days a week) 
 
Product Information
  
Product name:
 LUPEROX® P
 Synonyms: 
Peroxyester, t-butyl perbenzoate, tert-butyl peroxybenzoate 
Molecular formula:
 C11 H14 O3
 Chemical family:
 Organic peroxide - peroxyesters
 Product use: 
Initiator
  2. HAZARDS IDENTIFICATION
  Emergency Overview
  Color:
 Colourless to yellow.
 Physical state: 
liquid 
 Odor: unpleasant 
 CAUTION! 
ORGANIC PEROXIDE. 
HAZARDOUS DECO

### Extract SDS Section

In [25]:
Header1 = "1. PRODUCT AND COMPANY IDENTIFICATION"
Header2 = "2. HAZARDS IDENTIFICATION"

In [27]:
print(PdfText.count(Header1))
PdfText.count(Header2)

1


1

In [28]:
# Locate begin and end of section header

import re

count = 1
for m in re.finditer(Header1, PdfText):
         print(f'Occurence {count} : {m.start()}, {m.end()}\n')
         count+=1

Occurence 1 : 128, 165



In [59]:
def FindOccurences(Text, SubString):
    # Find all Occurences of SubString in text. Return dict with begin and end position in Text for each occurence
    # If not found, returns empty dictionary
    
    count = 1
    Dict = {}
    for m in re.finditer(SubString, Text):
         Dict[count] = (m.start(), m.end())
         count += 1
            
    return(Dict)

In [30]:
def VerifyOccurence(AllHeaders):
    # Check if SubString occurs in Text.
    
    if len(AllHeaders) == 0:
        s = "No Occurences"
    elif len(AllHeaders) > 1:
        s = "Multiple Occurences"
    else:
        s = "OK - Single Occurence"
    
    return(s)   

In [60]:
AllHeaders1 = FindOccurences(PdfText, Header1)

print(VerifyOccurence(AllHeaders1))
    
print(AllHeaders1)

OK - Single Occurence
{1: (128, 165)}


In [48]:
def ExtractText(Text, Begin, End):
    # Extract text Segment between 2 position indexes in a text string
    
    return(Text[Begin:End])

In [16]:
ExtractText(PdfText, 128, 165)

'1. PRODUCT AND COMPANY IDENTIFICATION'

In [61]:
# Locate begin and end of section "1. PRODUCT AND COMPANY IDENTIFICATION"
#
# Starts with the first postition after this header, ends with the last position before the next header

AllHeaders1 = FindOccurences(PdfText, Header1)
AllHeaders2 = FindOccurences(PdfText, Header2)

print(VerifyOccurence(AllHeaders1))
print(VerifyOccurence(AllHeaders2))

print(AllHeaders1[1][1], AllHeaders2[1][0])

OK - Single Occurence
OK - Single Occurence
165 785


In [41]:
Section1 = ExtractText(PdfText, AllHeaders1[1][1] + 1, AllHeaders2[1][0] - 1)

print(Section1)

  Company
  
Arkema Inc.
 900 First Avenue 
King of Prussia, Pennsylvania 19406 

 Functional Additives
  
Customer Service Telephone Number: 
(800) 331-7654 
(Monday through Friday, 8:30 AM to 5:30 PM EST) 
 Emergency Information
  
Transportation:
 CHEMTREC: (800) 424-9300 
(24 hrs., 7 days a week) 
Medical: 
Rocky Mountain Poison Center:
 (866) 767-5089 
(24 hrs., 7 days a week) 
 
Product Information
  
Product name:
 LUPEROX® P
 Synonyms: 
Peroxyester, t-butyl perbenzoate, tert-butyl peroxybenzoate 
Molecular formula:
 C11 H14 O3
 Chemical family:
 Organic peroxide - peroxyesters
 Product use: 
Initiator
 


### Section 0

The initial text that comes before section 1

In [49]:
# Extract Section before Header "1. PRODUCT AND COMPANY IDENTIFICATION"

Section0 = ExtractText(PdfText, 0, AllHeaders1[1][0] - 1)

print(Section0)

   Material Safety Data Sheet 
  LUPEROX® P
      Product code: 051000
   Version 2.0
 Issued on: 07/11/2011
 Page: 1 / 10
    


### Extract Entire PDF file


In [125]:
def extract_raw_text(file):

    pdfFileObj = open(file, "rb") 
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 

    raw_text = ""
    for n in range(pdfReader.numPages):
        raw_text += pdfReader.getPage(n).extractText()

    ##raw_text = raw_text.split("\n") 

    pdfFileObj.close() 
    return raw_text

In [52]:
PDFfile = extract_raw_text(InputFile)
print(type(PDFfile))

<class 'str'>


In [53]:
print(PDFfile.count(Header1))
PDFfile.count(Header2)

1


1

In [62]:
# Locate begin and end of section "1. PRODUCT AND COMPANY IDENTIFICATION"
#
# Starts with the first postition after this header, ends with the last position before the next header

AllHeaders1 = FindOccurences(PDFfile, Header1)
AllHeaders2 = FindOccurences(PDFfile, Header2)

print(VerifyOccurence(AllHeaders1))
print(VerifyOccurence(AllHeaders2))

print(AllHeaders1[1][1], AllHeaders2[1][0])

OK - Single Occurence
OK - Single Occurence
165 785


In [56]:
Section1 = ExtractText(PDFfile, AllHeaders1[1][1] + 1, AllHeaders2[1][0] - 1)

print(Section1)

  Company
  
Arkema Inc.
 900 First Avenue 
King of Prussia, Pennsylvania 19406 

 Functional Additives
  
Customer Service Telephone Number: 
(800) 331-7654 
(Monday through Friday, 8:30 AM to 5:30 PM EST) 
 Emergency Information
  
Transportation:
 CHEMTREC: (800) 424-9300 
(24 hrs., 7 days a week) 
Medical: 
Rocky Mountain Poison Center:
 (866) 767-5089 
(24 hrs., 7 days a week) 
 
Product Information
  
Product name:
 LUPEROX® P
 Synonyms: 
Peroxyester, t-butyl perbenzoate, tert-butyl peroxybenzoate 
Molecular formula:
 C11 H14 O3
 Chemical family:
 Organic peroxide - peroxyesters
 Product use: 
Initiator
 


In [57]:
# Extract Section before Header "1. PRODUCT AND COMPANY IDENTIFICATION"

Section0 = ExtractText(PDFfile, 0, AllHeaders1[1][0] - 1)

print(Section0)

   Material Safety Data Sheet 
  LUPEROX® P
      Product code: 051000
   Version 2.0
 Issued on: 07/11/2011
 Page: 1 / 10
    


### Create a dictionary with begin and end of each section


In [85]:
HeaderDict = { 0 : "", \
               1 : "1. PRODUCT AND COMPANY IDENTIFICATION", \
               2 : "2. HAZARDS IDENTIFICATION", \
               3 : "3. COMPOSITION/INFORMATION ON INGREDIENTS", \
               4 : "4. FIRST AID MEASURES", \
               5 : "5. FIRE-FIGHTING MEASURES", \
               6 : "6. ACCIDENTAL RELEASE MEASURES", \
               7 : "7. HANDLING AND STORAGE", \
               8 : "8. EXPOSURE CONTROLS/PERSONAL PROTECTION", \
               9 : "9. PHYSICAL AND CHEMICAL PROPERTIES", \
               10 : "10. STABILITY AND REACTIVITY", \
               11 : "11. TOXICOLOGICAL INFORMATION", \
               12 : "12. ECOLOGICAL INFORMATION", \
               13 : "13. DISPOSAL CONSIDERATIONS", \
               14 : "14. TRANSPORT INFORMATION", \
               15 : "15. REGULATORY INFORMATION", \
               16 : "16. OTHER INFORMATION" \
             }

In [86]:
HeaderDict[8]

'8. EXPOSURE CONTROLS/PERSONAL PROTECTION'

In [87]:
HeaderIndexes = {}

for Header in range(1,17):
    AllHeaders = FindOccurences(PDFfile, HeaderDict[Header])
    VerifyOccurence(AllHeaders)
    HeaderIndexes[Header] = AllHeaders[1]
    
print(HeaderDictIndexes)  
    


{1: (128, 165), 2: (785, 810), 3: (1697, 1738), 4: (2141, 2162), 5: (2643, 2668), 6: (4213, 4243), 7: (5228, 5251), 8: (6901, 6941), 9: (8890, 8925), 10: (9802, 9830), 11: (11251, 11280), 12: (12348, 12374), 13: (13280, 13307), 14: (14191, 14216), 15: (14691, 14717), 16: (17953, 17974)}


In [88]:
SectionIndexes = {}

SectionIndexes[0] = (0, HeaderIndexes[1][0] - 1)   # Section 0 is to contain PDF file intro

for Section in range(1,16):    
    SectionIndexes[Section] = (HeaderIndexes[Section][1] + 1, HeaderIndexes[Section + 1][0] - 1)
    
SectionIndexes[16] = (HeaderIndexes[16][1] + 1, len(PDFfile) )

print(SectionIndexes)

{0: (0, 127), 1: (166, 784), 2: (811, 1696), 3: (1739, 2140), 4: (2163, 2642), 5: (2669, 4212), 6: (4244, 5227), 7: (5252, 6900), 8: (6942, 8889), 9: (8926, 9801), 10: (9831, 11250), 11: (11281, 12347), 12: (12375, 13279), 13: (13308, 14190), 14: (14217, 14690), 15: (14718, 17952), 16: (17975, 19120)}


In [89]:
SectionText = {}

for Section in range(0,17):
    SectionText[Section] = ExtractText(PDFfile, SectionIndexes[Section][0], SectionIndexes[Section][1])

print(SectionText)

{0: '   Material Safety Data Sheet \n  LUPEROX® P\n      Product code: 051000\n   Version 2.0\n Issued on: 07/11/2011\n Page: 1 / 10\n    ', 1: '  Company\n  \nArkema Inc.\n 900 First Avenue \nKing of Prussia, Pennsylvania 19406 \n\n Functional Additives\n  \nCustomer Service Telephone Number: \n(800) 331-7654 \n(Monday through Friday, 8:30 AM to 5:30 PM EST) \n Emergency Information\n  \nTransportation:\n CHEMTREC: (800) 424-9300 \n(24 hrs., 7 days a week) \nMedical: \nRocky Mountain Poison Center:\n (866) 767-5089 \n(24 hrs., 7 days a week) \n \nProduct Information\n  \nProduct name:\n LUPEROX® P\n Synonyms: \nPeroxyester, t-butyl perbenzoate, tert-butyl peroxybenzoate \nMolecular formula:\n C11 H14 O3\n Chemical family:\n Organic peroxide - peroxyesters\n Product use: \nInitiator\n ', 2: '  Emergency Overview\n  Color:\n Colourless to yellow.\n Physical state: \nliquid \n Odor: unpleasant \n CAUTION! \nORGANIC PEROXIDE. \nHAZARDOUS DECOMPOSITION MAY OCCUR.\n  MAY CAUSE ALLERGIC SKIN

In [90]:
for Section in range(0,17):
    print("-------------\nSection ", Section, ": ", HeaderDict[Section], "\n-------------\n", SectionText[Section], "\n")


-------------
Section  0 :   
-------------
    Material Safety Data Sheet 
  LUPEROX® P
      Product code: 051000
   Version 2.0
 Issued on: 07/11/2011
 Page: 1 / 10
     

-------------
Section  1 :  1. PRODUCT AND COMPANY IDENTIFICATION 
-------------
   Company
  
Arkema Inc.
 900 First Avenue 
King of Prussia, Pennsylvania 19406 

 Functional Additives
  
Customer Service Telephone Number: 
(800) 331-7654 
(Monday through Friday, 8:30 AM to 5:30 PM EST) 
 Emergency Information
  
Transportation:
 CHEMTREC: (800) 424-9300 
(24 hrs., 7 days a week) 
Medical: 
Rocky Mountain Poison Center:
 (866) 767-5089 
(24 hrs., 7 days a week) 
 
Product Information
  
Product name:
 LUPEROX® P
 Synonyms: 
Peroxyester, t-butyl perbenzoate, tert-butyl peroxybenzoate 
Molecular formula:
 C11 H14 O3
 Chemical family:
 Organic peroxide - peroxyesters
 Product use: 
Initiator
  

-------------
Section  2 :  2. HAZARDS IDENTIFICATION 
-------------
   Emergency Overview
  Color:
 Colourless to yellow.

In [92]:
def PDF2Sections(PDFfile):
    # Convert a PDF file in a Dict of Sections

    # Locate all Headers:
    HeaderIndexes = {}

    for Header in range(1,17):
        AllHeaders = FindOccurences(PDFfile, HeaderDict[Header])
        VerifyOccurence(AllHeaders)
        HeaderIndexes[Header] = AllHeaders[1]        

    # Determine begin and end position of each header:    
    SectionIndexes = {}
                                    
    SectionIndexes[0] = (0, HeaderIndexes[1][0] - 1)   # Section 0 is to contain PDF file intro
    
    for Section in range(1,16):
        SectionIndexes[Section] = (HeaderIndexes[Section][1] + 1, HeaderIndexes[Section + 1][0] - 1)
    
    SectionIndexes[16] = (HeaderIndexes[16][1] + 1, len(PDFfile) )

    # Extract the text of each section:
    SectionText = {}
    
    for Section in range(0,17):
        SectionText[Section] = ExtractText(PDFfile, SectionIndexes[Section][0], SectionIndexes[Section][1])
    
    return(SectionText)
    

In [93]:
def PrintSections(SectionDict):
    # Print out the PDF converted to Dictionary

    for Section in range(0,17):
        print("-------------\nSection ", Section, ": ", HeaderDict[Section], "\n-------------\n", SectionDict[Section], "\n")
        
    return(None)


In [94]:
PDFSectionsText = PDF2Sections(PDFfile)
PrintSections(PDFSectionsText)

-------------
Section  0 :   
-------------
    Material Safety Data Sheet 
  LUPEROX® P
      Product code: 051000
   Version 2.0
 Issued on: 07/11/2011
 Page: 1 / 10
     

-------------
Section  1 :  1. PRODUCT AND COMPANY IDENTIFICATION 
-------------
   Company
  
Arkema Inc.
 900 First Avenue 
King of Prussia, Pennsylvania 19406 

 Functional Additives
  
Customer Service Telephone Number: 
(800) 331-7654 
(Monday through Friday, 8:30 AM to 5:30 PM EST) 
 Emergency Information
  
Transportation:
 CHEMTREC: (800) 424-9300 
(24 hrs., 7 days a week) 
Medical: 
Rocky Mountain Poison Center:
 (866) 767-5089 
(24 hrs., 7 days a week) 
 
Product Information
  
Product name:
 LUPEROX® P
 Synonyms: 
Peroxyester, t-butyl perbenzoate, tert-butyl peroxybenzoate 
Molecular formula:
 C11 H14 O3
 Chemical family:
 Organic peroxide - peroxyesters
 Product use: 
Initiator
  

-------------
Section  2 :  2. HAZARDS IDENTIFICATION 
-------------
   Emergency Overview
  Color:
 Colourless to yellow.

In [129]:
InputFile = 'data/LC15750.pdf'

In [130]:
PDFfile = extract_raw_text(InputFile)
print(type(PDFfile), len(PDFfile))
print(PDFfile)

<class 'str'> 29000
 Isopropyl Alcohol (2
-Propanol)
 Safety Data Sheet
    according to Federal Register / Vol. 77,
 No. 58 / Monday, March 26, 2012 / Rules and Regulations
 Date of issue
: 11/14/2013
 Revision date
: 01/23/2020
 Supersedes
: 01/26/2018
 Version
: 1.3
  01/23/2020
 EN (English US)
  Page 1
  SECTION 1
: Identification
  1.1.
 Identification
 Product form
 : Substance
 Substance name
 : Isopropyl Alcohol (
2-Propanol)
 CAS
-No.
 : 67-63-0 Product code
 : LC15750
 Formula
 : C3H8O
 Synonyms
 : 1-methylethanol / 1
-methylethyl alcohol / 2
-hydroxypropane / 
dimethyl carbinol / ethyl carbinol / 
hydroxypropane / IPA / i
-propanol / isoethylcarbinol / propan
-2-ol / sec
-propanol
  1.2.
 Recommended use and restrictions on use
 Use of the substance/mixture
 : Disinfectant
 Solvent
 Recommended use
 : Laboratory chemicals
 Restrictions on use
 : Not for food, drug or household use
  1.3.
 Supplier
 LabChem, Inc.
 Jackson's Pointe Commerce Park Building 1000, 1010 Jackson's 

In [131]:
HeaderIndexes = {}

for Header in range(1,17):
    AllHeaders = FindOccurences(PDFfile, HeaderDict[Header])
    if len(AllHeaders):
        VerifyOccurence(AllHeaders)
        HeaderIndexes[Header] = AllHeaders[1]
    else:
        print(HeaderDict[Header], ": not found")

1. PRODUCT AND COMPANY IDENTIFICATION : not found
2. HAZARDS IDENTIFICATION : not found
3. COMPOSITION/INFORMATION ON INGREDIENTS : not found
4. FIRST AID MEASURES : not found
5. FIRE-FIGHTING MEASURES : not found
6. ACCIDENTAL RELEASE MEASURES : not found
7. HANDLING AND STORAGE : not found
8. EXPOSURE CONTROLS/PERSONAL PROTECTION : not found
9. PHYSICAL AND CHEMICAL PROPERTIES : not found
10. STABILITY AND REACTIVITY : not found
11. TOXICOLOGICAL INFORMATION : not found
12. ECOLOGICAL INFORMATION : not found
13. DISPOSAL CONSIDERATIONS : not found
14. TRANSPORT INFORMATION : not found
15. REGULATORY INFORMATION : not found
16. OTHER INFORMATION : not found


In [106]:
InputFile = 'data/Carbowax Sentry PEG 400 NF - Dow - 2015-03-26.pdf'

In [111]:
PDFfile = extract_raw_text(InputFile)
print(type(PDFfile), len(PDFfile))
print(PDFfile)

<class 'str'> 22993
®
 

 
Trademark of The Dow Chemical Company ("Dow") or an affiliated 
company of Dow
 
Page
 
1
 
of
 
10
 
 
 
SAFETY
 
DATA SHEET
 
THE DOW CHEMICAL COMPANY
 
 
Product name:
 
CARBOWAX

 
SENTRY

 
POLYETHYLENE 
GLYCOL 400 NF, FCC GRADE; MACROGOL 400 Ph. Eur.
 
Issue Date:
 
03/26/2015
 
Print Date:
 
04/01/2015
 
 
THE DOW CHEMICAL COMPANY
 
encourages and expects you to read and understand the entire 
(M)SDS, as there is important information throughout the document.  We expect you to follow the 
precautions identified in this document unless your use conditions would necessitate other appropri
ate 
methods or actions.
 
 
1. IDENTIFICATION
 
Product name:
 
CARBOWAX

 
SENTRY

 
POLYETHYLENE GLYCOL 400 NF, FCC GRADE; 
MACROGOL 400 Ph. Eur.
 
 
Recommended use of the chemical and restrictions on use
 
Identified uses:
 

 
Product Line, a partial list of applications include 
pharmaceutical products, personal care products, automotive products, household produ

In [119]:
HeaderIndexes = {}

for Header in range(1,17):
    AllHeaders = FindOccurences(PDFfile, HeaderDict[Header])
    if len(AllHeaders):
        VerifyOccurence(AllHeaders)
        HeaderIndexes[Header] = AllHeaders[1]
    else:
        print(HeaderDict[Header], ": not found")

1. PRODUCT AND COMPANY IDENTIFICATION : not found
2. HAZARDS IDENTIFICATION : not found
3. COMPOSITION/INFORMATION ON INGREDIENTS : not found
4. FIRST AID MEASURES : not found
5. FIRE-FIGHTING MEASURES : not found
6. ACCIDENTAL RELEASE MEASURES : not found
7. HANDLING AND STORAGE : not found
8. EXPOSURE CONTROLS/PERSONAL PROTECTION : not found
9. PHYSICAL AND CHEMICAL PROPERTIES : not found
10. STABILITY AND REACTIVITY : not found
11. TOXICOLOGICAL INFORMATION : not found
12. ECOLOGICAL INFORMATION : not found
13. DISPOSAL CONSIDERATIONS : not found
14. TRANSPORT INFORMATION : not found
15. REGULATORY INFORMATION : not found
16. OTHER INFORMATION : not found


In [124]:
InputFile = 'data/gylcol-ether-dpm-sds.pdf'

In [126]:
PDFfile = extract_raw_text(InputFile)
print(type(PDFfile), len(PDFfile))
print(PDFfile)

<class 'str'> 0

